In [5]:
import json
import requests

In [6]:
# Porijõgi station code
station_code = 'SJA3124000'

In [3]:
# Read JSON file and get a list with a single dictionary
with open('data/estmodel20211101100645.json', encoding='utf-8') as file:
    data = json.load(file)

In [3]:
# POST request using the JSON data
r = requests.post('https://estmodel.envir.ee/', json=data[0])

In [4]:
r.status_code

200

In [9]:
# Test writing a JSON string to file
with open('data/{}_response.json'.format(station_code), 'w', encoding='utf-8') as file:
    json_string = json.dumps(r.json(), ensure_ascii=False, indent=4)
    print(json_string, file=file)
    file.close()

Tried creating a function for recursively looking through the JSON object and getting to where the Porijõgi station data is located at. The info is hidden somewhere inside a nested structure of dictionaries and lists, so to make the procedure more robust and usable regardless of the level of the station there needs to be ways to switch the search target between a dictionary and list.

In [97]:
# def extract_json(json_object, station_code):
#     if isinstance(json_object, dict) and json_object:
#         for k, v in json_object.items():
#             if json_object.get('code') == station_code:
#                 print('Station name: {}'.format(json_object['name']))
#                 print('Station code: {}'.format(station_code))
#                 factor_dict = json_object['adjustments'][0]
#                 print('Adjustment factor for parameter {}: {}'.format(factor_dict.get('parameter').upper(), factor_dict.get('factor')))
#                 for i in range(len(json_object['diffuseSources'])):
#                     if json_object['diffuseSources'][i]['type'] in ['arableland', 'forest', 'pasture', 'water']:
#                         print(json_object['diffuseSources'][i]['type'])
#                         print(json_object['diffuseSources'][i]['estimates'])
#                 print('\n')
#             else:
#                 extract_json(v, station_code)
#     elif isinstance(json_object, list) and json_object:
#         for item in json_object:
#             extract_json(item, station_code)

Currently the function prints out the estimates of the model per LULC type 14 times, so obviously something is wrong with the loop. However, extracting the estimates for atmospheric deposition, etc from these should not be too difficult. The estimates could be written into a CSV file or something like that.

The more difficult part is writing the new adjustment factor into a JSON file in the correct position, save that as a new JSON or simply a dictionary in memory and submit it as a POST request.

Test updating the adjustment factor of JSON file and saving as a new file.

In [36]:
with open('data/estmodel20211102072608.json', encoding='utf-8') as file:
    data = json.load(file)

In [94]:
# Get adjustments for station
def get_adjustments(data, station_code):
    if isinstance(data, dict):
        for k, v in data.items():
            if k == 'code' and 'year' not in data.keys():
                if v == station_code:
                    adjustments = data.get('adjustments')
                    print(adjustments)
            else:
                get_adjustments(v, station_code)
    elif isinstance(data, list):
        for i in range(len(data)):
            get_adjustments(data[i], station_code)

In [95]:
# Get diffuse sources for station
def get_diffuse_sources(data, station_code):
    if isinstance(data, dict):
        for k, v in data.items():
            if k == 'code' and 'year' not in data.keys():
                if v == station_code:
                    diffuse_sources = data.get('diffuseSources')
                    print(diffuse_sources)
            else:
                get_diffuse_sources(v, station_code)
    elif isinstance(data, list):
        for i in range(len(data)):
            get_diffuse_sources(data[i], station_code)

In [96]:
get_adjustments(data, station_code)

[{'parameter': 'tn', 'factor': 1.1}, {'parameter': 'tp', 'factor': 0.9}]


In [93]:
get_diffuse_sources(data, station_code)

[{'code': None, 'id': None, 'name': 'Porijõgi: Reola – Vana-Kuuste tee (Uhti)', 'description': None, 'adjustments': [], 'estimates': [{'parameter': 'tn', 'anthropogenicDischarge': 0, 'atmosphericDeposition': 0.13739853448970385, 'naturalDischarge': 149.76170562355773, 'retention': 81.11430933090568}, {'parameter': 'tp', 'anthropogenicDischarge': 0, 'atmosphericDeposition': 0.0025788030206979565, 'naturalDischarge': 4.890938517304428, 'retention': 2.5034817214681357}], 'type': None, 'area': 1.145825, 'drainedArea': 0.0528169794779904, 'harvestedArea': 0, 'claySoilArea': 0, 'fertileSoilArea': 0.7647821219831703, 'peatSoilArea': 0, 'scatteredPopulation': 0, 'configurations': [], 'fertilizers': []}, {'code': None, 'id': None, 'name': 'Porijõgi: Reola – Vana-Kuuste tee (Uhti)', 'description': None, 'adjustments': [], 'estimates': [{'parameter': 'tn', 'anthropogenicDischarge': 42731.630588040345, 'atmosphericDeposition': 14.332421164742346, 'naturalDischarge': 15622.057741145984, 'retention'